# 03 Office class

In [9]:
print 'xx'

xx


In [10]:
import time
import collections
#import worker
import server
import medareda_lib
from datetime import timedelta

import urllib3
urllib3.disable_warnings()

import worker_status

#import work

In [11]:
class Office(object):
    def __init__(self,work):
        self.work = work
        self.basename = "%s-%s" %('medaredaworker',self.work)
        name = ''
    
    
    def server_states(self):
        return server.get_server_states()

    def countServers(self):
        return server.countServers(self.basename)
    
    def countWorkers(self):
        return worker_status.countWorkers(self.basename)
        
    def addWorker(self):        
        image_name = '%s-i' %self.basename
        name = server.createServer(self.basename, image_name)
        worker_status.addWorker(name)       
    
    def removeWorker(self): # deleteServer
        delete_name = server.delete(self.basename)
        print 'DELETED', delete_name
        worker_status.removeWorker(delete_name)

        
    def getProcessQueueLength(self):
        conn = medareda_lib.get_conn()
        cur = conn.cursor()
        sql = "SELECT count(*) from iPrice where status = 'wait' "
        cur.execute(sql)
        results = cur.fetchall()
        conn.commit()
        conn.close()
        count = results[0][0]
        return count
    
    
    def getProcessQueueTime(self):
        conn = medareda_lib.get_conn()
        cur = conn.cursor()
        sql = "SELECT min(date),max(date) from iPrice where status = 'wait' "
        cur.execute(sql)
        results = cur.fetchall()
        conn.commit()
        conn.close()
        print results
        min_date = results[0][0]
        max_date = results[0][1] # could use now
        
        if max_date == None: # ( no waiting rows)
            return timedelta(seconds=0)
        
        queue_time = max_date - min_date
        return queue_time
    
        
    def reasonToStartServer(self):
        if self.countServers() > 20: # max servers
            return False
        if self.countServers() < 2:
            print 'TO FEW WORKERS'
            return True
        
        queue_length = self.getProcessQueueLength()
        if queue_length > 100:  # TODO max queue lenght
            print 'QUEUE LENGHT TO LONG', queue_length
            return True
        
        if self.getProcessQueueTime() > timedelta(seconds=30):
            print "PROCESSING TO SLOW"
            return True
        return False
            
        
    def reasonToStopServer(self):
        if self.countServers() < 3 : # min workers 2
            self.False
            
        if self.getProcessQueueLength() < self.countWorkers():
            print "MORE SERVERS THAN WORK"
            return True
        
        #if self.getProcessQueueTime() > 1: # min
        #    return True
        
        return False
    
    def manageWorkers(self):
        # get queue length (work)
        # get max time in queue (work)
        # get number of workers building
        # get number of workers working
        
        if self.reasonToStartServer():  # work > 100 or time in queue > 1hr
            self.addWorker()
        elif self.reasonToStopServer(): # que < servers * 2
            time.sleep(5) 
            if self.reasonToStopServer():
                self.removeWorker()
        # else:
        #    time.sleep(5)
        print 'x'
        pass
    
    def shutDown(self):
        #cs = self.get_cs()
        print 'SHUTDOWN TODO'
        return
        for server in  pyrax.cloudservers.list():
            server_name = server.name
            if server_name.startswith(self.basename):
                if server.status == 'ACTIVE':
                    print 'Delete', server_name
                    server.delete()
            #else:
            #    print 'leave', server.name
            

test

In [33]:
work = 'iprice'
print 'start'

def test():
    
    office = Office(work)

    count = office.countWorkers()
    print 'Worker count =', count

    office.addWorker()
    #office.removeWorker()

    #office.getAmountOfWork() # queue lenght * weight

    # for 1 hour, while time < now + 1hr

    #print office.getProcessQueueTime()
    #office.manageWorkers()

    #office.shutDown()

test()


start
host 95.138.170.160
Worker count =

/Library/Python/2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning
/Library/Python/2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning


 0
create new server, called medaredaworker-iprice-2
from image medaredaworker-iprice-i

/Library/Python/2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning
/Library/Python/2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning



create server ...

/Library/Python/2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning
/Library/Python/2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning



Adding woker status
host 95.138.170.160


In [8]:
import mock
import time

#class workerTestCase(unittest.TestCase):
@mock.patch('server._create_server')
@mock.patch('server._delete')
def test_office(c,d):
    #w = worker.Worker1('iPrice')
    #time.sleep(5)
    #w.delete()

    #def test_office(office):
    office = Office('iPrice')
    print 'Pre Number of servers in office', office.countServers()    
    print 'Pre Number of workers in office =', office.countWorkers() 
    #office.addWorker()
    #time.sleep(5)
    office.removeWorker()
    print 'Post Number of servers in office', office.countServers()
    print 'Post Number of workers in office =', office.countWorkers()


test_office()


/Library/Python/2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning
/Library/Python/2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning


Pre Number of servers in office 0
Pre Number of workers in office = host 95.138.170.160
9

/Library/Python/2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning
/Library/Python/2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning



No server found to delete
DELETED None
Removing worker staus
host 95.138.170.160
host 95.138.170.160
Post Number of servers in office

/Library/Python/2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning
/Library/Python/2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning


 0
Post Number of workers in office = host 95.138.170.160
9
